In [13]:
import os
import numpy as np
import pandas as pd
import sklearn as skl
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [14]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import (
    RidgeCV,
    MultiTaskLassoCV,
    MultiTaskElasticNetCV,
    LogisticRegressionCV,
)
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [15]:
from xgboost import XGBClassifier

In [16]:
datapath = os.path.join("data", "US", "us_data_weekly.csv")
dus_weekly = pd.read_csv(datapath, index_col=0)
dus_weekly = dus_weekly.dropna()

### A) Creating new features

In [17]:
def add_ts_features(df, cols, max_lag=20, windows=[10, 20, 50]):
    all_features = {}

    for col in cols:
        y = df[col]

        # --- Rolling statistics ---
        for w in windows:
            roll = y.rolling(w)
            all_features[f"{col}_mean_{w}"] = roll.mean()
            all_features[f"{col}_std_{w}"] = roll.std()
            all_features[f"{col}_q25_{w}"] = roll.quantile(0.25)
            all_features[f"{col}_q75_{w}"] = roll.quantile(0.75)
            all_features[f"{col}_q05_{w}"] = roll.quantile(0.05)
            all_features[f"{col}_q90_{w}"] = roll.quantile(0.9)
            all_features[f"{col}_range_{w}"] = roll.max() - roll.min()

            # Z-score et momentum
            all_features[f"{col}_zscore_{w}"] = (y - roll.mean()) / roll.std()
            all_features[f"{col}_momentum_{w}"] = y - y.shift(w)

        # Ratio de moyennes rapides / lentes
        all_features[f"{col}_ratio_{10}_{50}"] = (
            y.rolling(10).mean() / y.rolling(50).mean()
        )

        # Volatilité annualisée approx
        all_features[f"{col}_vol_20"] = y.rolling(20).std() * np.sqrt(52)

        # --- Lags bruts ---
        for lag in [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50]:
            all_features[f"{col}_lag_{lag}"] = y.shift(lag - 1)

        # --- Autocorrélations (in-sample) ---

        acf_vals = acf(y.dropna(), nlags=max_lag, fft=True)
        for lag in range(1, max_lag + 1):
            all_features[f"{col}_autocorr_{lag}"] = acf_vals[lag]

    # --- Construction finale ---
    features = pd.DataFrame(all_features, index=df.index)
    return features


In [18]:
cols = [
    "DGS1MO",
    "DGS3MO",
    "DGS6MO",
    "DGS1",
    "DGS2",
    "DGS3",
    "DGS5",
    "DGS7",
    "DGS10",
    "DGS20",
    "DGS30",
]

dus_features = add_ts_features(dus_weekly, cols)
dus_features = dus_features.dropna()
dus_weekly = dus_weekly.merge(
    dus_features, how="inner", left_index=True, right_index=True
)

In [19]:
for col in cols:
    dus_weekly[f"Y_{col}"] = (dus_weekly[col] > 0).astype(int).shift(-1)


dus_weekly = dus_weekly.dropna()
dus_weekly = dus_weekly.replace([np.inf, -np.inf], np.nan)
dus_weekly = dus_weekly.ffill()

# we can now remove the original yield columns
dus_weekly = dus_weekly.drop(columns=cols)

In [20]:
Yw = dus_weekly[[f"Y_{col}" for col in cols]]
Xw = dus_weekly.drop(columns=[f"Y_{col}" for col in cols])

print(f"there are {Xw.shape[1]} features in the dataset")

there are 738 features in the dataset


We augmented the dataset by adding a very large amount of features created from the yields time series. however, many of these features are not informative so we need to remove them before training our model. 

We'll filter features by keeping only those with a mutual information score above than a given threshold. We'll create several datasets of features containing features filtered for the thresholds 0.03, 0.035, 0.04, 0.05 and train models on these specific datasets. 

In [21]:
window_train = 52 * 15
window_pred = 4

alphas = np.logspace(-5, 2, 30)
tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "logreg",
            MultiOutputClassifier(
                LogisticRegressionCV(
                    penalty="l2",
                    cv=tscv,
                    Cs=alphas,
                    fit_intercept=False,
                    scoring="accuracy",
                    max_iter=5000,
                )
            ),
        ),
    ]
)

pipepca = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.99)),
        (
            "logreg",
            MultiOutputClassifier(
                LogisticRegressionCV(
                    penalty="l2",
                    cv=tscv,
                    Cs=alphas,
                    fit_intercept=False,
                    scoring="accuracy",
                    max_iter=5000,
                )
            ),
        ),
    ]
)

We train logistic regression models for the 4 datasets containing features with mutual information larger than 0.03, 0.035, 0.04, 0.05 respectively. Then we save the results in datasets. 

The dataset with variables with mutual information > 0.03 contains 266 features, which is very high compared to the numer of points in the training datasets (52*15 lines), so we'll also train a model that performs a PCA before applying the logistic regression in order to reduce the number of features and see if it can improve the performance.

In [23]:
initial_number_of_features = Xw.shape[1]

threshold_list = [0.03, 0.035, 0.04, 0.05]

datasets = {t: pd.DataFrame() for t in threshold_list}

for threshold_mi in tqdm(threshold_list):
    selected_features_w = set()
    for col in Yw.columns:
        mi = mutual_info_classif(Xw, Yw[col])
        top_features_i = Xw.columns[mi > threshold_mi]
        selected_features_w.update(top_features_i)

    datasets[threshold_mi] = Xw[list(selected_features_w)]

    print(
        f"we removed {initial_number_of_features - len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}"
    )
    print(
        f"Number of variables in the dataset with MI threshold {threshold_mi}:",
        datasets[threshold_mi].shape[1],
    )
    print(
        f"Macro and market variables in the dataset with MI threshold {threshold_mi}:",
        [col for col in datasets[threshold_mi].columns if "DGS" not in col],
    )

 25%|██▌       | 1/4 [00:09<00:28,  9.34s/it]

we removed 474 features in the weekly dataset for threshold 0.03
Number of variables in the dataset with MI threshold 0.03: 264
Macro and market variables in the dataset with MI threshold 0.03: ['UNRATE', 'MTSDS133FMS', 'FGEXPND', 'M2SL', 'HOUST', 'IRLTLT01JPM156N', 'M1SL', 'IRLTLT01DEM156N', 'DTCTHFNM', 'IRLTLT01GBM156N', 'CUUR0000SA0L2', 'IRLTLT01AUM156N', 'FYGFDPUN', 'CPIULFSL', 'T10YIE', 'CPIAUCSL', 'GFDEGDQ188S']


 50%|█████     | 2/4 [00:18<00:18,  9.29s/it]

we removed 559 features in the weekly dataset for threshold 0.035
Number of variables in the dataset with MI threshold 0.035: 179
Macro and market variables in the dataset with MI threshold 0.035: ['UNRATE', 'HOUST', 'GFDEBTN', 'IRLTLT01DEM156N', 'M2REAL', 'DTCTHFNM', 'USTPU', 'AAA', 'IRLTLT01FRM156N', 'IRLTLT01AUM156N', 'IRLTLT01CAM156N', 'DNDGRG3M086SBEA', 'BAA']


 75%|███████▌  | 3/4 [00:27<00:09,  9.27s/it]

we removed 628 features in the weekly dataset for threshold 0.04
Number of variables in the dataset with MI threshold 0.04: 110
Macro and market variables in the dataset with MI threshold 0.04: ['IRLTLT01JPM156N', 'PCEPI', 'SRVPRD']


100%|██████████| 4/4 [00:37<00:00,  9.28s/it]

we removed 700 features in the weekly dataset for threshold 0.05
Number of variables in the dataset with MI threshold 0.05: 38
Macro and market variables in the dataset with MI threshold 0.05: ['UNRATE', 'USCONS']


## C) XGBoost

We'll now train XGboost models to see whether it can capture non linearities in the data and improve accuracy of out-of-sample predictions. Once again, we'll train some models on the 4 datasets that we created. 

For the first dataset (with features with a MI>0.03), we'll also train a model that performs a PCA to reduce the number of features before applying XGboost. 

The pipelines that we'll use are the following:

In [24]:
window_train = 52 * 15
window_pred = 4
tscv = TimeSeriesSplit(n_splits=4)


# parameters to test in cross validation
param_grid = {
    "xgb__estimator__n_estimators": [100, 200],
    "xgb__estimator__learning_rate": [0.01, 0.05],
    "xgb__estimator__max_depth": [4, 7],
    "xgb__estimator__subsample": [0.5, 0.7],
    "xgb__estimator__colsample_bytree": [0.4, 0.8],
    "xgb__estimator__min_child_weight": [5, 10],
    "xgb__estimator__reg_alpha": [0.5, 1.0],
    "xgb__estimator__reg_lambda": [5, 10],
}


# pipeline without PCA
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "xgb",
            MultiOutputClassifier(
                XGBClassifier(
                    objective="binary:logistic",
                    use_label_encoder=False,
                    n_jobs=-1,
                    random_state=42,
                    verbosity=0,
                )
            ),
        ),
    ]
)


# pipeline with PCA
pipepca = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.99)),
        (
            "xgb",
            MultiOutputClassifier(
                XGBClassifier(
                    objective="binary:logistic",
                    use_label_encoder=False,
                    n_jobs=-1,
                    random_state=42,
                    verbosity=0,
                )
            ),
        ),
    ]
)


# and we'll use gridsearchCV to cross validate the model:
GridSearchCV(
    pipepca,  # or pipe
    param_grid,
    cv=tscv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=0,
)


,estimator,"Pipeline(step...None, ...)))])"
,param_grid,"{'xgb__estimator__colsample_bytree': [0.4, 0.8], 'xgb__estimator__learning_rate': [0.01, 0.05], 'xgb__estimator__max_depth': [4, 7], 'xgb__estimator__min_child_weight': [5, 10], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


We now train XGBoost models on the datasets:

# AUDRIC FAIS TOURNER LE BLOC DE CODE CI DESSOUS STP (+ le bloc juste au dessus aussi pour que ca fonctionne)

In [26]:
for threshold in [0.035]:
    print(
        f"training model on dataset with features with mutual information above {threshold}"
    )

    Xw = datasets[threshold]

    # for the dataset containing features with mutual information above 0.03:
    # we'll train our xgboost model with and without PCA since there are many features in this dataset
    if threshold == 0.03:
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(
            range(0, len(Xw) - window_train - window_pred + 1, window_pred)
        ):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(
                pipe, param_grid, cv=tscv, scoring="accuracy", n_jobs=-1, verbose=0
            )

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps["xgb"].estimators_[i]  # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)

        acc = pd.DataFrame(accuracies, columns=Yw.columns)
        params = pd.DataFrame(params_by_target, columns=Yw.columns)
        feature_imp = pd.DataFrame(feature_importance, columns=Yw.columns)
        Y_true_flat = np.array(y_true).reshape(
            -1, np.array(y_true).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(
            Y_true_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )
        Y_pred_flat = np.array(y_pred).reshape(
            -1, np.array(y_pred).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(
            Y_pred_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )

        print(
            f"For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:"
        )
        print(acc.mean())
        acc.to_csv("results of models/accuracies xgboost, 15y train test.csv")
        params.to_csv("results of models/params xgboost, 15y train test.csv")
        feature_imp.to_csv(
            "results of models/feature importance xgboost, 15y train test.csv"
        )
        ypred.to_csv("results of models/forecast xgboost, 15y train test.csv")
        ytrue.to_csv("results of models/true values xgboost, 15y train test.csv")

        #### Now we do the same training but this time we add the PCA in the pipeline before training the model
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(
            range(0, len(Xw) - window_train - window_pred + 1, window_pred)
        ):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(
                pipepca, param_grid, cv=tscv, scoring="accuracy", n_jobs=-1, verbose=0
            )

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps["xgb"].estimators_[i]  # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)

        acc = pd.DataFrame(accuracies, columns=Yw.columns)
        params = pd.DataFrame(params_by_target, columns=Yw.columns)
        feature_imp = pd.DataFrame(feature_importance, columns=Yw.columns)
        Y_true_flat = np.array(y_true).reshape(
            -1, np.array(y_true).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(
            Y_true_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )
        Y_pred_flat = np.array(y_pred).reshape(
            -1, np.array(y_pred).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(
            Y_pred_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )

        print(
            f"For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} and PCA is:"
        )
        print(acc.mean())
        acc.to_csv("results of models/accuracies xgboost, pca, 15y train test.csv")
        params.to_csv("results of models/params xgboost, pca, 15y train test.csv")
        feature_imp.to_csv(
            "results of models/feature importance xgboost, pca, 15y train test.csv"
        )
        ypred.to_csv("results of models/forecast xgboost, pca, 15y train test.csv")
        ytrue.to_csv("results of models/true values xgboost, pca, 15y train test.csv")

    # for other datasets with larger mutual information threshold, we don't apply PCA and train the model directly
    else:
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(
            range(0, len(Xw) - window_train - window_pred + 1, window_pred)
        ):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(
                pipe, param_grid, cv=tscv, scoring="accuracy", n_jobs=-1, verbose=0
            )

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps["xgb"].estimators_[i]  # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)

        acc = pd.DataFrame(accuracies, columns=Yw.columns)
        params = pd.DataFrame(params_by_target, columns=Yw.columns)
        feature_imp = pd.DataFrame(feature_importance, columns=Yw.columns)
        Y_true_flat = np.array(y_true).reshape(
            -1, np.array(y_true).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(
            Y_true_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )
        Y_pred_flat = np.array(y_pred).reshape(
            -1, np.array(y_pred).shape[2]
        )  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(
            Y_pred_flat, index=Yw.iloc[780 : Yw.shape[0] - 1].index, columns=Yw.columns
        )

        print(
            f"For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:"
        )
        print(acc.mean())
        acc.to_csv(
            f"results of models/accuracies xgboost, MI {threshold}, 15y train test.csv"
        )
        params.to_csv(
            f"results of models/params xgboost, MI {threshold}, 15y train test.csv"
        )
        feature_imp.to_csv(
            f"results of models/feature importance xgboost, MI {threshold}, 15y train test.csv"
        )
        ypred.to_csv(
            f"results of models/forecast xgboost, MI {threshold}, 15y train test.csv"
        )
        ytrue.to_csv(
            f"results of models/true values xgboost, MI {threshold}, 15y train test.csv"
        )


training model on dataset with features with mutual information above 0.035


  0%|          | 0/79 [00:00<?, ?it/s]Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x1410903b0>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/tf/lib/python3.12/site-packages/xgboost/core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 
  0%|          | 0/79 [01:55<?, ?it/s]


KeyboardInterrupt: 